# BASIC BUY SELL ALGORITHMS

## 🚨 Disclaimer

> **⚠️ This project is strictly for educational use only.**
>
> - It is not intended for live or real-money trading.
> - It does not provide financial or investment advice.
> - The developer of this repository is not responsible for any financial loss resulting from the use or misuse of this code.
>
> Use this bot only in test environments or simulations. **Do not connect to real wallets or trading accounts.**
>
> This application does not provide investment advice. All data, models, functions and forecasts are for educational and analytical purposes only. Do not use this information to make investment decisions. All investment decisions are the responsibility of the user.
>
> **WARNING:** This application and all included analyses, data, and forecasts are strictly for educational and analytical purposes only. **Do not use for investment purposes under any circumstances.** The owner of the code assumes no responsibility for any financial, investment, or trading decisions made based on the information provided. **All responsibility lies with the user.**

> **ATTENTION:** In Turkey, investment transactions must only be carried out through intermediary institutions and financial organizations that are authorized and licensed by the Capital Markets Board of Turkey (CMB). This application or code should not be used for investment transactions and must not be considered for investment purposes.

### ⚠️ Yasal Uyarı (Türkçe)

❗️Bu proje **yatırım tavsiyesi değildir**.  
📌 Bu proje sadece **eğitim** amaçlı geliştirilmiştir.

- Herhangi bir finansal ya da yatırım tavsiyesi sunmaz.  
- Bu kodun kullanımından doğabilecek finansal kayıplardan geliştirici sorumlu değildir.  
- Bu bot sadece test ortamlarında veya simülasyonlarda kullanılmalıdır. Gerçek cüzdanlar veya ticaret hesapları ile **bağlantı kurmayınız**.
> Bu uygulama yatırım tavsiyesi içermez. Sunulan veriler, modeller, fonksiyonlar ve tahminler yalnızca eğitim ve analiz amaçlıdır. Yatırım kararlarınızı bu verilerle almanız önerilmez. Her türlü yatırım kararı kullanıcıya aittir.
>
> **UYARI:** Bu uygulama ve içerdiği tüm analizler, veriler ve tahminler yalnızca eğitim ve analiz amaçlıdır. **Kesinlikle yatırım amaçlı kullanmayınız.** Uygulamada yer alan bilgilerle yapılacak herhangi bir yatırım, alım-satım veya finansal işlemden doğacak her türlü sorumluluk kullanıcıya aittir. **Kodun sahibi hiçbir şekilde sorumluluk kabul etmez.**

> **DİKKAT:** Türkiye’de yatırım işlemleri yalnızca Sermaye Piyasası Kurulu (SPK) tarafından yetkilendirilmiş ve lisanslanmış aracı kurumlar ve finansal kuruluşlar aracılığıyla yapılmalıdır. Bu uygulama veya kod, herhangi bir yatırım işlemi için kullanılmamalıdır ve yatırım amacıyla değerlendirilmemelidir.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = pd.read_csv("btc_data_5min.csv")

In [ ]:
data.columns

Index(['timestamp', 'sum_volume_last_5min', 'sum_quote_volume_last_5min',
       'sum_trades_last_5min', 'sum_taker_base_last_5min',
       'sum_taker_quote_last_5min', 'open', 'high', 'low', 'close', 'volume',
       'quote_asset_volume', 'num_trades', 'taker_buy_base', 'taker_buy_quote',
       'year', 'month', 'day', 'hour', 'minute'],
      dtype='object')

In [ ]:
five_month = data[data['timestamp'] >= '2025-01-01'].copy()

In [ ]:
two_years = data[data['timestamp'] >= '2023-05-18'].copy()

In [ ]:
one_week = data[data['timestamp'] >= '2025-05-01'].copy()

# SMA

In [ ]:
def sma_crossover_backtest(df, fee_rate=0.001, initial_balance=100):
    """
    SMA Crossover (5-20) stratejisi: stepwise (bar-by-bar) backtest.
    """


    df = df.copy().reset_index(drop=True)

    # SMA hesaplamaları
    df['SMA_5'] = df['close'].rolling(window=5).mean()
    df['SMA_20'] = df['close'].rolling(window=20).mean()

    # Al-Sat sinyalleri
    df['signal'] = 0
    df.loc[
        (df['SMA_5'] > df['SMA_20']) & (df['SMA_5'].shift(1) <= df['SMA_20'].shift(1)),
        'signal'
    ] = 1  # AL
    df.loc[
        (df['SMA_5'] < df['SMA_20']) & (df['SMA_5'].shift(1) >= df['SMA_20'].shift(1)),
        'signal'
    ] = -1  # SAT

    balance = initial_balance
    fee_paid = 0.0
    position = 0  # 0: no position, 1: long
    entry_price = 0.0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:
            # Alış
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:
            # Satış
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        # Güncel pozisyona göre equity
        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        # Kayıt
        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final, fee, result_df = sma_crossover_backtest(five_month, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 12.32 USDT
Toplam Fee: 94.36 USDT
Toplam İşlem Sayısı: 2328


In [ ]:
final, fee, result_df = sma_crossover_backtest(two_years, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 0.00 USDT
Toplam Fee: 102.71 USDT
Toplam İşlem Sayısı: 13657


In [ ]:
final, fee, result_df = sma_crossover_backtest(one_week, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 90.17 USDT
Toplam Fee: 13.87 USDT
Toplam İşlem Sayısı: 150


# EMA

In [ ]:
def ema_crossover_backtest(df, fee_rate=0.001, initial_balance=1000):
    """
    EMA Crossover (9-21) stratejisi adım adım işleyen backtest fonksiyonu.
    """


    df = df.copy().reset_index(drop=True)

    # EMA hesapla
    df['EMA_9'] = df['close'].ewm(span=9, adjust=False).mean()
    df['EMA_21'] = df['close'].ewm(span=21, adjust=False).mean()

    # Sinyal üretimi (cross durumları)
    df['signal'] = 0
    df.loc[(df['EMA_9'] > df['EMA_21']) & (df['EMA_9'].shift(1) <= df['EMA_21'].shift(1)), 'signal'] = 1  # AL
    df.loc[(df['EMA_9'] < df['EMA_21']) & (df['EMA_9'].shift(1) >= df['EMA_21'].shift(1)), 'signal'] = -1  # SAT

    balance = initial_balance
    position = 0  # 0: none, 1: long
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        # Güncel equity hesabı
        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final, fee, result_df = ema_crossover_backtest(five_month, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 21.97 USDT
Toplam Fee: 82.08 USDT
Toplam İşlem Sayısı: 1685


In [ ]:
final, fee, result_df = ema_crossover_backtest(two_years, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 0.01 USDT
Toplam Fee: 102.68 USDT
Toplam İşlem Sayısı: 9937


In [ ]:
final, fee, result_df = ema_crossover_backtest(one_week, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 95.93 USDT
Toplam Fee: 9.54 USDT
Toplam İşlem Sayısı: 99


In [ ]:
def volume_spike_strategy(df, volume_col='sum_volume_last_5min', lookback=20,
                          spike_threshold=2.0, holding_period=5,
                          fee_rate=0.001, initial_balance=1000):
    """
    Hacim patlaması stratejisi (bar-bar simülasyon).
    """
    import numpy as np
    import pandas as pd

    df = df.copy().reset_index(drop=True)
    df['volume_avg'] = df[volume_col].rolling(window=lookback).mean()

    balance = initial_balance
    fee_paid = 0.0
    position = 0  # 0: none, 1: in position
    entry_price = 0
    exit_timer = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(lookback, len(df)):  # İlk lookback kadar bar atlanır
        row = df.iloc[i]
        signal = 0

        # Spike tespiti ve pozisyona giriş
        if position == 0 and row[volume_col] > row['volume_avg'] * spike_threshold:
            signal = 1  # AL
            entry_price = row['close']
            position = 1
            exit_timer = holding_period
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee

        # Pozisyondan çıkış
        elif position == 1:
            exit_timer -= 1
            if exit_timer == 0:
                pnl = (row['close'] - entry_price) / entry_price
                gross_balance = balance * (1 + pnl)
                fee = gross_balance * fee_rate
                balance = gross_balance - fee
                fee_paid += fee
                position = 0

        # Güncel varlık (equity) hesabı
        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        # Kayıt
        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # İlk lookback kadar satırı doldurmak için None ekle
    prepend = [None] * lookback
    df['signal'] = prepend + signal_list
    df['position'] = prepend + position_list
    df['equity'] = prepend + equity_list
    df['fee_total'] = prepend + fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s == 1)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final, fee, result_df = volume_spike_strategy(
    five_month,
    volume_col='sum_volume_last_5min',
    lookback=20,
    spike_threshold=2.0,
    holding_period=5,
    fee_rate=0.001,
    initial_balance=100
)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 4.02 USDT
Toplam Fee: 102.05 USDT
Toplam İşlem Sayısı: 1626


In [ ]:
final, fee, result_df = volume_spike_strategy(
    two_years,
    volume_col='sum_volume_last_5min',
    lookback=20,
    spike_threshold=2.0,
    holding_period=5,
    fee_rate=0.001,
    initial_balance=100
)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 0.00 USDT
Toplam Fee: 101.83 USDT
Toplam İşlem Sayısı: 8802


In [ ]:
final, fee, result_df = volume_spike_strategy(
    one_week,
    volume_col='sum_volume_last_5min',
    lookback=20,
    spike_threshold=2.0,
    holding_period=5,
    fee_rate=0.001,
    initial_balance=100
)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 85.20 USDT
Toplam Fee: 19.30 USDT
Toplam İşlem Sayısı: 105


# RSI

In [ ]:
def rsi_strategy_stepwise(df, period=14, lower=30, upper=70,
                          fee_rate=0.001, initial_balance=1000):
    """
    RSI tabanlı stratejinin adım adım (stepwise) işleyen, düzeltilmiş versiyonu.
    """

    df = df.copy().reset_index(drop=True)

    # RSI hesaplama
    delta = df['close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=period).mean()
    avg_loss = pd.Series(loss).rolling(window=period).mean()
    rs = avg_gain / (avg_loss + 1e-10)
    df['RSI'] = 100 - (100 / (1 + rs))

    # Simülasyon değişkenleri
    balance = initial_balance
    position = 0  # 1: long, 0: none
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(period, len(df)):  # RSI oluşmaya başladıktan sonra
        row = df.iloc[i]

        signal = 0

        if row['RSI'] < lower and position == 0:
            signal = 1  # AL
            fee = balance * fee_rate
            fee_paid += fee
            balance -= fee  # Komisyon düş
            entry_price = row['close']
            position = 1

        elif row['RSI'] > upper and position == 1:
            signal = -1  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            fee_paid += fee
            balance = gross_balance - fee  # Kar/zarar uygula ve fee düş
            position = 0

        # Güncel pozisyona göre equity hesapla
        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        equity_list.append(equity)
        position_list.append(position)
        signal_list.append(signal)
        fee_list.append(fee_paid)

    # İlk period kadar olan kısımlar NaN olacak şekilde doldur
    prepend_nan = [None] * period
    df['signal'] = prepend_nan + signal_list
    df['position'] = prepend_nan + position_list
    df['equity'] = prepend_nan + equity_list
    df['fee_total'] = prepend_nan + fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final, fee, result_df = rsi_strategy_stepwise(
    one_week,
    period=14,
    lower=30,
    upper=70,
    fee_rate=0.001,
    initial_balance=100
)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 98.65 USDT
Toplam Fee: 4.50 USDT
Toplam İşlem Sayısı: 46


In [ ]:
final, fee, result_df = rsi_strategy_stepwise(
    five_month,
    period=14,
    lower=30,
    upper=70,
    fee_rate=0.001,
    initial_balance=100
)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 45.31 USDT
Toplam Fee: 50.99 USDT
Toplam İşlem Sayısı: 782


In [ ]:
final, fee, result_df = rsi_strategy_stepwise(
    two_years,
    period=14,
    lower=30,
    upper=70,
    fee_rate=0.001,
    initial_balance=100
)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 1.44 USDT
Toplam Fee: 107.98 USDT
Toplam İşlem Sayısı: 4822


# MACD

In [ ]:
def macd_zero_cross_backtest(df, fee_rate=0.001, initial_balance=1000):
    """
    MACD Zero Cross stratejisi backtest fonksiyonu.
    - AL: MACD 0'ın altından yukarı keserse.
    - SAT: MACD 0'ın üstünden aşağı keserse.
    """



    df = df.copy().reset_index(drop=True)

    # MACD hesapla
    df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()
    df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA_12'] - df['EMA_26']

    # Sinyal üretimi (zero cross)
    df['signal'] = 0
    df.loc[(df['MACD'] > 0) & (df['MACD'].shift(1) <= 0), 'signal'] = 1   # AL
    df.loc[(df['MACD'] < 0) & (df['MACD'].shift(1) >= 0), 'signal'] = -1  # SAT

    balance = initial_balance
    position = 0  # 0: none, 1: long
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        # Güncel equity hesabı
        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = macd_zero_cross_backtest(one_week, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 97.93 USDT
Toplam Fee: 7.60 USDT
Toplam İşlem Sayısı: 77


In [ ]:
final_balance, total_fee, result_df = macd_zero_cross_backtest(five_month, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 34.64 USDT
Toplam Fee: 76.61 USDT
Toplam İşlem Sayısı: 1287


In [ ]:
final_balance, total_fee, result_df = macd_zero_cross_backtest(two_years, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 0.15 USDT
Toplam Fee: 108.90 USDT
Toplam İşlem Sayısı: 7713


# Bollinger

In [ ]:
def bollinger_reversion_backtest(df, window=20, num_std=2, fee_rate=0.001, initial_balance=1000):
    """
    Bollinger Bands Mean Reversion stratejisi backtest fonksiyonu (long-only).
    - AL: Fiyat alt bandı aşağıdan yukarıya keserse.
    - SAT: Fiyat üst bandı yukarıdan aşağıya keserse.
    """


    df = df.copy().reset_index(drop=True)

    # Bollinger Bands hesapla
    df['MA'] = df['close'].rolling(window).mean()
    df['STD'] = df['close'].rolling(window).std()
    df['Upper'] = df['MA'] + num_std * df['STD']
    df['Lower'] = df['MA'] - num_std * df['STD']

    # Sinyal üretimi
    df['signal'] = 0

    # AL: Alt bandı aşağıdan yukarıya keserse
    df.loc[
        (df['close'] > df['Lower']) & (df['close'].shift(1) <= df['Lower'].shift(1)),
        'signal'
    ] = 1

    # SAT: Üst bandı yukarıdan aşağıya keserse
    df.loc[
        (df['close'] < df['Upper']) & (df['close'].shift(1) >= df['Upper'].shift(1)),
        'signal'
    ] = -1

    balance = initial_balance
    position = 0  # 0: none, 1: long
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = bollinger_reversion_backtest(one_week, window=20, num_std=2, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 97.01 USDT
Toplam Fee: 4.14 USDT
Toplam İşlem Sayısı: 114


In [ ]:
final_balance, total_fee, result_df = bollinger_reversion_backtest(five_month, window=20, num_std=2, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 45.63 USDT
Toplam Fee: 51.40 USDT
Toplam İşlem Sayısı: 2057


In [ ]:
final_balance, total_fee, result_df = bollinger_reversion_backtest(two_years, window=20, num_std=2, fee_rate=0.001, initial_balance=100)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 2.89 USDT
Toplam Fee: 109.72 USDT
Toplam İşlem Sayısı: 11565


# MA Slope

In [ ]:
def ma_slope_backtest(df, ma_window=14, fee_rate=0.001, initial_balance=100):
    """
    Moving Average Slope stratejisi backtest fonksiyonu.
    - AL: MA eğimi negatiften pozitife dönerse
    - SAT: MA eğimi pozitiften negatife dönerse
    """


    df = df.copy().reset_index(drop=True)

    # SMA hesapla
    df['MA'] = df['close'].rolling(ma_window).mean()
    # Eğimi hesapla (mevcut MA - bir önceki MA)
    df['slope'] = df['MA'] - df['MA'].shift(1)

    # Sinyal üretimi
    df['signal'] = 0
    # Slope negatiften pozitife geçiyorsa AL
    df.loc[(df['slope'] > 0) & (df['slope'].shift(1) <= 0), 'signal'] = 1
    # Slope pozitiften negatife geçiyorsa SAT
    df.loc[(df['slope'] < 0) & (df['slope'].shift(1) >= 0), 'signal'] = -1

    balance = initial_balance
    position = 0
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = ma_slope_backtest(one_week, ma_window=14)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 77.35 USDT
Toplam Fee: 24.59 USDT
Toplam İşlem Sayısı: 286


In [ ]:
final_balance, total_fee, result_df = ma_slope_backtest(five_month, ma_window=14)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 0.95 USDT
Toplam Fee: 98.07 USDT
Toplam İşlem Sayısı: 4687


In [ ]:
final_balance, total_fee, result_df = ma_slope_backtest(two_years, ma_window=14)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 0.00 USDT
Toplam Fee: 104.77 USDT
Toplam İşlem Sayısı: 27140


#Quote Volume Divergence

In [ ]:
def quote_volume_divergence_backtest(
    df, lookback=10, fee_rate=0.001, initial_balance=100
):
    """
    Quote Volume Divergence stratejisi için basit backtest fonksiyonu (long only).
    - AL: Fiyat yeni dipte ama quote_volume yeni dipte değilse (pozitif diverjans)
    - SAT: Fiyat yeni tepe ama quote_volume yeni tepe değilse (negatif diverjans)
    """

    df = df.copy().reset_index(drop=True)
    # Sinyal sütunu
    df['signal'] = 0

    for i in range(lookback, len(df)):
        price_window = df['close'][i - lookback : i]
        vol_window = df['quote_asset_volume'][i - lookback : i]
        # Fiyat yeni dip, volume yeni dip değilse = AL
        if (
            df['close'][i] == price_window.min()
            and df['quote_asset_volume'][i] > vol_window.min()
        ):
            df.at[i, 'signal'] = 1
        # Fiyat yeni tepe, volume yeni tepe değilse = SAT
        elif (
            df['close'][i] == price_window.max()
            and df['quote_asset_volume'][i] < vol_window.max()
        ):
            df.at[i, 'signal'] = -1

    balance = initial_balance
    position = 0
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = quote_volume_divergence_backtest(one_week, lookback=10)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 102.55 USDT
Toplam Fee: 0.40 USDT
Toplam İşlem Sayısı: 6


In [ ]:
final_balance, total_fee, result_df = quote_volume_divergence_backtest(five_month, lookback=10)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 103.01 USDT
Toplam Fee: 1.35 USDT
Toplam İşlem Sayısı: 22


In [ ]:
final_balance, total_fee, result_df = quote_volume_divergence_backtest(two_years, lookback=10)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 267.17 USDT
Toplam Fee: 27.15 USDT
Toplam İşlem Sayısı: 343


# Buy Pressure Dominance

In [ ]:
def buy_pressure_dominance_backtest(
    df, ratio_threshold=0.7, fee_rate=0.001, initial_balance=100
):
    """
    Buy Pressure Dominance stratejisi için backtest fonksiyonu (long only).
    - AL: Alıcı baskısı oranı (taker_buy_quote / quote_asset_volume) threshold'u aşarsa ve artıyorsa
    - SAT: Alıcı baskısı oranı threshold altına inerse veya hızlıca düşerse
    """

    df = df.copy().reset_index(drop=True)

    # Buy pressure ratio hesapla
    df['buy_pressure_ratio'] = df['taker_buy_quote'] / df['quote_asset_volume']

    # Sinyal sütunu
    df['signal'] = 0

    for i in range(1, len(df)):
        ratio_now = df.at[i, 'buy_pressure_ratio']
        ratio_prev = df.at[i - 1, 'buy_pressure_ratio']

        # AL sinyali: oran threshold üzerinde ve artıyorsa
        if (ratio_now > ratio_threshold) and (ratio_now > ratio_prev):
            df.at[i, 'signal'] = 1

        # SAT sinyali: oran threshold altına inerse veya önceki bara göre ciddi düşüş varsa
        elif (ratio_now < ratio_threshold) and (ratio_now < ratio_prev * 0.95):
            df.at[i, 'signal'] = -1

    balance = initial_balance
    position = 0
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = buy_pressure_dominance_backtest(one_week, ratio_threshold=0.7)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 44.95 USDT
Toplam Fee: 56.13 USDT
Toplam İşlem Sayısı: 1494


In [ ]:
final_balance, total_fee, result_df = buy_pressure_dominance_backtest(five_month, ratio_threshold=0.7)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 0.00 USDT
Toplam Fee: 97.32 USDT
Toplam İşlem Sayısı: 22890


In [ ]:
final_balance, total_fee, result_df = buy_pressure_dominance_backtest(two_years, ratio_threshold=0.7)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 0.00 USDT
Toplam Fee: 101.49 USDT
Toplam İşlem Sayısı: 128305


# Break of High Volume

In [ ]:
def break_of_high_volume_confirm_backtest(
    df, lookback=20, fee_rate=0.001, initial_balance=100
):
    """
    Break of High with Volume Confirm stratejisi için backtest fonksiyonu (long only).
    - AL: Fiyat, son 'lookback' barın en yükseğini yukarı kırarsa VE hacim, son 'lookback' ortalamasının üstünde ise
    - SAT: Fiyat, son 'lookback' barın en düşüğünü aşağı kırarsa (stop)
    """

    df = df.copy().reset_index(drop=True)
    df['signal'] = 0

    for i in range(lookback, len(df)):
        high_window = df['high'][i-lookback:i]
        low_window = df['low'][i-lookback:i]
        volume_window = df['volume'][i-lookback:i]

        # AL: Yeni en yüksek + hacim ortalama üstü
        if (
            df['close'][i] > high_window.max()
            and df['volume'][i] > volume_window.mean()
        ):
            df.at[i, 'signal'] = 1

        # SAT: Yeni en düşük
        elif df['close'][i] < low_window.min():
            df.at[i, 'signal'] = -1

    balance = initial_balance
    position = 0
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = break_of_high_volume_confirm_backtest(one_week, lookback=20)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 97.29 USDT
Toplam Fee: 5.81 USDT
Toplam İşlem Sayısı: 294


In [ ]:
final_balance, total_fee, result_df = break_of_high_volume_confirm_backtest(five_month, lookback=20)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 49.79 USDT
Toplam Fee: 63.48 USDT
Toplam İşlem Sayısı: 4795


In [ ]:
final_balance, total_fee, result_df = break_of_high_volume_confirm_backtest(two_years, lookback=20)

Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 1.05 USDT
Toplam Fee: 110.40 USDT
Toplam İşlem Sayısı: 26026


# Trade Count Spike

In [ ]:
def trade_count_spike_backtest(
    df, lookback=20, spike_multiplier=2.0, fee_rate=0.001, initial_balance=100
):
    """
    Trade Count Spike stratejisi için backtest fonksiyonu (long only).
    - AL: num_trades, son 'lookback' ortalamasının 'spike_multiplier' katını aşarsa
    - SAT: num_trades tekrar ortalamanın altına inerse veya başka bir çıkış kuralı varsa
    """
    df = df.copy().reset_index(drop=True)
    df['signal'] = 0

    for i in range(lookback, len(df)):
        trades_window = df['num_trades'][i - lookback : i]
        trades_mean = trades_window.mean()
        current_trades = df['num_trades'][i]

        # AL: Spike tespit edilirse
        if current_trades > trades_mean * spike_multiplier:
            df.at[i, 'signal'] = 1

        # SAT: Tekrar ortalamanın altına inerse (ve pozisyon açıksa)
        elif current_trades < trades_mean and df['signal'][i-1] != 0:
            df.at[i, 'signal'] = -1

    balance = initial_balance
    position = 0
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = trade_count_spike_backtest(one_week, lookback=20, spike_multiplier=2.0)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 92.50 USDT
Toplam Fee: 15.06 USDT
Toplam İşlem Sayısı: 447


In [ ]:
final_balance, total_fee, result_df = trade_count_spike_backtest(five_month, lookback=20, spike_multiplier=2.0)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 24.24 USDT
Toplam Fee: 83.26 USDT
Toplam İşlem Sayısı: 4936


In [ ]:
final_balance, total_fee, result_df = trade_count_spike_backtest(two_years, lookback=20, spike_multiplier=2.0)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 0.03 USDT
Toplam Fee: 111.56 USDT
Toplam İşlem Sayısı: 30491


# Doji Candle Reversal

In [ ]:
def doji_candle_reversal_backtest(
    df, doji_thresh=0.1, fee_rate=0.001, initial_balance=100
):
    """
    Doji Candle Reversal backtest fonksiyonu (long only).
    - AL: Downtrend + doji + ardından yeşil mum
    - SAT: Pozisyon açıkken doji + ardından kırmızı mum
    doji_thresh: Doji tespiti için gövdenin toplam bara oranı eşiği (örn: 0.1)
    """
    df = df.copy().reset_index(drop=True)
    df['body'] = (df['close'] - df['open']).abs()
    df['range'] = (df['high'] - df['low']).replace(0, 1e-9)
    df['is_doji'] = (df['body'] / df['range']) < doji_thresh

    df['signal'] = 0

    for i in range(2, len(df)):
        # AL: Önceki bar doji, ondan önceki kırmızı (düşüş), mevcut bar yeşil
        if (
            df['is_doji'][i-1]
            and df['close'][i-2] < df['open'][i-2]  # önceki düşüş
            and df['close'][i] > df['open'][i]      # mevcut yeşil
        ):
            df.at[i, 'signal'] = 1
        # SAT: Pozisyon açıkken, önceki bar doji, ondan önceki yeşil (yükseliş), mevcut bar kırmızı
        elif (
            df['is_doji'][i-1]
            and df['close'][i-2] > df['open'][i-2]  # önceki yükseliş
            and df['close'][i] < df['open'][i]      # mevcut kırmızı
        ):
            df.at[i, 'signal'] = -1

    balance = initial_balance
    position = 0
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = doji_candle_reversal_backtest(one_week, doji_thresh=0.1)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 96.15 USDT
Toplam Fee: 3.86 USDT
Toplam İşlem Sayısı: 89


In [ ]:
final_balance, total_fee, result_df = doji_candle_reversal_backtest(five_month, doji_thresh=0.1)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 56.60 USDT
Toplam Fee: 53.17 USDT
Toplam İşlem Sayısı: 1249


In [ ]:
final_balance, total_fee, result_df = doji_candle_reversal_backtest(two_years, doji_thresh=0.1)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 5.55 USDT
Toplam Fee: 125.57 USDT
Toplam İşlem Sayısı: 7328


# VWAP

In [ ]:
def vwap_reversion_backtest(
    df, deviation=0.01, fee_rate=0.001, initial_balance=100
):
    """
    VWAP Reversion stratejisi için backtest fonksiyonu (long only).
    - AL: Fiyat VWAP'ın 'deviation' oranı altına düştükten sonra tekrar üstüne çıkarsa (geri dönüş)
    - SAT: Fiyat VWAP'ın 'deviation' oranı üstüne çıktıktan sonra tekrar altına inerse
    """
    df = df.copy().reset_index(drop=True)

    # VWAP hesapla
    df['cum_vol_x_price'] = (df['close'] * df['volume']).cumsum()
    df['cum_volume'] = df['volume'].cumsum()
    df['VWAP'] = df['cum_vol_x_price'] / df['cum_volume']

    df['signal'] = 0

    for i in range(1, len(df)):
        vwap = df.at[i, 'VWAP']
        close_now = df.at[i, 'close']
        close_prev = df.at[i-1, 'close']
        vwap_prev = df.at[i-1, 'VWAP']

        # AL: Fiyat, VWAP'ın belirli bir oran altından tekrar üstüne çıkarsa
        if (
            close_now > vwap * (1 - deviation)
            and close_prev <= vwap_prev * (1 - deviation)
        ):
            df.at[i, 'signal'] = 1
        # SAT: Fiyat, VWAP'ın belirli bir oran üstünden tekrar altına inerse
        elif (
            close_now < vwap * (1 + deviation)
            and close_prev >= vwap_prev * (1 + deviation)
        ):
            df.at[i, 'signal'] = -1

    balance = initial_balance
    position = 0
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = vwap_reversion_backtest(one_week, deviation=0.01)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 101.00 USDT
Toplam Fee: 0.20 USDT
Toplam İşlem Sayısı: 32


In [ ]:
final_balance, total_fee, result_df = vwap_reversion_backtest(five_month, deviation=0.01)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 95.34 USDT
Toplam Fee: 1.01 USDT
Toplam İşlem Sayısı: 129


In [ ]:
final_balance, total_fee, result_df = vwap_reversion_backtest(two_years, deviation=0.01)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 107.81 USDT
Toplam Fee: 1.46 USDT
Toplam İşlem Sayısı: 113


# Openin Range

In [ ]:
def opening_range_breakout_backtest(
    df, opening_range=5, fee_rate=0.001, initial_balance=100
):
    """
    Opening Range Breakout (ORB) stratejisi için backtest fonksiyonu (long only).
    - opening_range: Açılış aralığını belirleyen bar sayısı (örn: ilk 5 bar)
    - AL: Fiyat açılış aralığının üstüne çıkar ve önceki bara göre üstte kapanırsa
    - SAT: Fiyat açılış aralığının altına düşerse (stop mantığı)
    """
    df = df.copy().reset_index(drop=True)
    df['signal'] = 0

    # Açılış aralığını belirle
    opening_high = df['high'][:opening_range].max()
    opening_low = df['low'][:opening_range].min()

    # Sinyal üretimi
    for i in range(opening_range, len(df)):
        # AL: Fiyat açılış yüksekliğini yukarı kırarsa ve önceki bara göre yukarıda kapanırsa
        if (
            df['close'][i] > opening_high
            and df['close'][i-1] <= opening_high
        ):
            df.at[i, 'signal'] = 1
        # SAT: Fiyat açılış düşük seviyesinin altına düşerse (stop)
        elif df['close'][i] < opening_low:
            df.at[i, 'signal'] = -1

    balance = initial_balance
    position = 0
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Açılış Aralığı: İlk {opening_range} bar")
    print(f"Opening High: {opening_high:.2f}, Opening Low: {opening_low:.2f}")
    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = opening_range_breakout_backtest(one_week, opening_range=5)


Açılış Aralığı: İlk 5 bar
Opening High: 94331.05, Opening Low: 94130.43
Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 102.52 USDT
Toplam Fee: 2.14 USDT
Toplam İşlem Sayısı: 134


In [ ]:
final_balance, total_fee, result_df = opening_range_breakout_backtest(five_month, opening_range=5)


Açılış Aralığı: İlk 5 bar
Opening High: 93866.00, Opening Low: 93536.52
Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 88.34 USDT
Toplam Fee: 4.67 USDT
Toplam İşlem Sayısı: 17381


In [ ]:
final_balance, total_fee, result_df = opening_range_breakout_backtest(two_years, opening_range=5)


Açılış Aralığı: İlk 5 bar
Opening High: 27452.35, Opening Low: 27367.19
Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 292.81 USDT
Toplam Fee: 4.02 USDT
Toplam İşlem Sayısı: 23237


In [ ]:
def quote_to_base_ratio_spike_backtest(
    df, lookback=20, spike_multiplier=1.5, fee_rate=0.001, initial_balance=100
):
    """
    Quote-to-Base Ratio Spike stratejisi için backtest fonksiyonu (long only).
    - AL: quote-to-base ratio son 'lookback' ortalamasının 'spike_multiplier' katını aşarsa
    - SAT: Oran tekrar ortalamanın altına inerse
    """

    df = df.copy().reset_index(drop=True)
    df['qtb_ratio'] = df['quote_asset_volume'] / df['volume'].replace(0, 1e-9)
    df['signal'] = 0

    for i in range(lookback, len(df)):
        ratio_window = df['qtb_ratio'][i-lookback:i]
        ratio_mean = ratio_window.mean()
        ratio_now = df['qtb_ratio'][i]

        # AL: Spike var
        if ratio_now > ratio_mean * spike_multiplier:
            df.at[i, 'signal'] = 1
        # SAT: Geriye döndü
        elif ratio_now < ratio_mean:
            df.at[i, 'signal'] = -1

    balance = initial_balance
    position = 0
    entry_price = 0
    fee_paid = 0

    equity_list = []
    position_list = []
    signal_list = []
    fee_list = []

    for i in range(len(df)):
        row = df.iloc[i]
        signal = row['signal']

        if signal == 1 and position == 0:  # AL
            entry_price = row['close']
            fee = balance * fee_rate
            balance -= fee
            fee_paid += fee
            position = 1

        elif signal == -1 and position == 1:  # SAT
            pnl = (row['close'] - entry_price) / entry_price
            gross_balance = balance * (1 + pnl)
            fee = gross_balance * fee_rate
            balance = gross_balance - fee
            fee_paid += fee
            position = 0

        equity = balance
        if position == 1:
            equity = balance * (row['close'] / entry_price)

        signal_list.append(signal)
        position_list.append(position)
        equity_list.append(equity)
        fee_list.append(fee_paid)

    # Pozisyon açık kaldıysa son fiyattan kapat
    if position == 1:
        row = df.iloc[-1]
        pnl = (row['close'] - entry_price) / entry_price
        gross_balance = balance * (1 + pnl)
        fee = gross_balance * fee_rate
        balance = gross_balance - fee
        fee_paid += fee
        equity = balance
        equity_list[-1] = equity

    df['sim_signal'] = signal_list
    df['position'] = position_list
    df['equity'] = equity_list
    df['fee_total'] = fee_list

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {equity_list[-1]:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print(f"Toplam İşlem Sayısı: {sum(1 for s in signal_list if s != 0)}")

    return equity_list[-1], fee_paid, df


In [ ]:
final_balance, total_fee, result_df = quote_to_base_ratio_spike_backtest(one_week, lookback=20, spike_multiplier=1.5)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 100.00 USDT
Toplam Fee: 0.00 USDT
Toplam İşlem Sayısı: 1063


In [ ]:
final_balance, total_fee, result_df = quote_to_base_ratio_spike_backtest(five_month, lookback=20, spike_multiplier=1.5)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 100.00 USDT
Toplam Fee: 0.00 USDT
Toplam İşlem Sayısı: 17959


In [ ]:
final_balance, total_fee, result_df = quote_to_base_ratio_spike_backtest(two_years, lookback=20, spike_multiplier=1.5)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 100.00 USDT
Toplam Fee: 0.00 USDT
Toplam İşlem Sayısı: 101226


# HOLD

In [ ]:
def hold_strategy_backtest(df, fee_rate=0.001, initial_balance=100):
    """
    Hold (Buy and Hold) stratejisi backtest fonksiyonu.
    Başlangıçta al, hiç satma, en sona kadar tut.
    """
    df = df.copy().reset_index(drop=True)

    # İlk alım
    entry_price = df.iloc[0]['close']
    fee = initial_balance * fee_rate
    balance = initial_balance - fee
    fee_paid = fee

    # En sonda elde edilen değer
    final_price = df.iloc[-1]['close']
    final_equity = balance * (final_price / entry_price)

    # Tüm df için equity sütunu oluşturalım
    df['position'] = 1
    df['sim_signal'] = 0
    df['fee_total'] = fee_paid
    df['equity'] = balance * (df['close'] / entry_price)

    print(f"Başlangıç Bakiye: {initial_balance:.2f} USDT")
    print(f"Final Bakiye: {final_equity:.2f} USDT")
    print(f"Toplam Fee: {fee_paid:.2f} USDT")
    print("Toplam İşlem Sayısı: 1 (al ve tut)")

    return final_equity, fee_paid, df


In [ ]:
final_balance, total_fee, result_df = hold_strategy_backtest(one_week)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 108.79 USDT
Toplam Fee: 0.10 USDT
Toplam İşlem Sayısı: 1 (al ve tut)


In [ ]:
final_balance, total_fee, result_df = hold_strategy_backtest(five_month)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 109.42 USDT
Toplam Fee: 0.10 USDT
Toplam İşlem Sayısı: 1 (al ve tut)


In [ ]:
final_balance, total_fee, result_df = hold_strategy_backtest(two_years)


Başlangıç Bakiye: 100.00 USDT
Final Bakiye: 373.62 USDT
Toplam Fee: 0.10 USDT
Toplam İşlem Sayısı: 1 (al ve tut)
